In [196]:
netlist = """
***************************************
* auCdl Netlist
*
* Library Name: ML_DATA_LIB
* View Name : asdsa
* Top Cewll Name : sadasdsdf
* Netlisted On: FGerv, asdfasfd, 2 10:35:01 2022
***************************************
.INCLUDE $calibre_source_added_place
*.BIPOLAR
*.REST = 2000
*.MEGA
.PARAM

*.EQUIV xcvr_avdd_clk = n12
*+    loaddely = n13
*+    rst_n = n14
.SUBCKT BUFFER A Z gnd gnds vdd vdds
+ n1 n2 n3
+ n4 n5
* net net0 = nnnn
* net Z = /zzzz/asdasd00
M1 Z net0 vdd vdds mypmos w=5 l=1
* net A = /gggg/asdsa
M2 net0 A vdd vdds mypmos w=5 l=1
+ area=122 fin=34
+ sde=44
M3 Z net0 gnd gnds mynmos w=2 l=1
M4 net0 A gnd gnds mynmos w=2 l=1
.ENDS

I1 net0 net1 0 gnds! vdd! vdds! BUFFER
I2 net1 net2 0 gnds! vdd! vdds! INVERTER
"""

In [399]:
import pyparsing as _p
import netlist as nl
net_dict = dict()

In [303]:
def handle_instance(token):
    inst = token.instance
    name = inst[0]
    pins, reference, params = divide_body(inst.instbody)
    parameters = dict_params(params)
    i = nl.instance(name, pins, reference, parameters)
    return [i]

In [304]:
def divide_body(objs):
    out = len(objs)
    try:
        out = next(idx for idx, obj in enumerate(objs) if "=" in obj)
    except StopIteration:
        pass
    return objs[:out-1], objs[out-1], objs[out:]

In [305]:
def dict_params(params):
    m_split = list(map(lambda x: x.split("="), params))
    key = map(lambda x: x[0], m_split)
    value = map(lambda x:x[1], m_split)
    return dict(zip(key, value))

In [326]:
def handle_ext_nets(token):
    inst = token.ext_nets
    net_dict[inst.key] = inst.value 
    return [{inst.key : inst.value}]

In [338]:
def handle_int_nets(token):
    inst = token.int_nets
    net_dict[inst.key] = inst.value 
    return [{inst.key : inst.value}]

In [390]:
def handle_subcircuit(token):
    sc = token.subcircuit
    nets = sc.nets
    name = sc.name
    subcontent = sc.subnetlist
    instances = list(filter(lambda x: type(x) == nl.instance, subcontent))
    int_nets = list(filter(lambda x: type(x) == dict, subcontent))
    s = nl.subcircuit(name, nets, instances)
    return [int_nets, s]

In [391]:
def parse_spectre(netlist_string):
    # newlines are part of the grammar, thus redifine the whitespaces without it
    ws = ' \t'
    _p.ParserElement.setDefaultWhitespaceChars(ws)

    # spectre netlist grammar definition
    EOL = _p.LineEnd().suppress() # end of line
    linebreak = _p.Suppress(_p.LineEnd() + '+') # breaking a line with backslash newline
    identifier=_p.Word(_p.alphanums+'_!<>/') # a name for...
    number=_p.Word(_p.nums + ".") # a number
    expression = _p.Word(_p.alphanums+'._*+-/()=!<>')
    net = expression # a net
    nets = _p.Group(_p.OneOrMore(net | linebreak)) # many nets
    cktname = identifier # name of a subcircuit
    cktname_end = _p.Keyword(".ENDS").suppress()
    instname = identifier
    instance = _p.Group(instname('instname') + nets('instbody') + EOL).setResultsName('instance')
    int_net = _p.Suppress("* net") + identifier('key') + _p.Suppress("=") + identifier('value') + EOL
    int_nets = _p.Group(int_net).setResultsName('int_nets')
    subcircuit_content = _p.Group(_p.ZeroOrMore(instance | int_nets | EOL)).setResultsName('subnetlist')
    cir_net = identifier
    cir_nets = _p.Group(_p.OneOrMore(cir_net('net') | linebreak))
    ext_net = _p.Suppress(_p.Literal("*.EQUIV") | _p.Literal("*+"))  + identifier('value') + _p.Suppress("=") + identifier('key') + EOL
    ext_nets = _p.Group(ext_net).setResultsName('ext_nets')
    subcircuit = _p.Group(
        # matches subckt <name> <nets> <newline>
        _p.Keyword(".SUBCKT").suppress() + cktname('name') + cir_nets('nets') + EOL  
        # matches the content of the subcircuit
        + subcircuit_content
        # matches ends <name> <newline>
        + cktname_end + EOL).setResultsName('subcircuit')
    include = _p.Group(_p.Keyword(".INCLUDE") + ... + _p.Keyword(".PARAM"))
    star_line = _p.Group(_p.Combine(_p.Keyword("***") + ... + _p.Keyword("***")))
    netlist_element = subcircuit | instance |_p.Suppress(include) | _p.Suppress(star_line) | ext_nets | EOL
    netlist = _p.ZeroOrMore(netlist_element) + _p.StringEnd()
    
    # parameters.setParseAction(handle_parameters)
    ext_nets.setParseAction(handle_ext_nets)
    int_nets.setParseAction(handle_int_nets)
    instance.setParseAction(handle_instance)
    subcircuit.setParseAction(handle_subcircuit)

    return netlist.parseString(netlist_string);

In [392]:
result1 = parse_spectre(netlist)

In [393]:
result1

ParseResults([{'n12': 'xcvr_avdd_clk'}, {'n13': 'loaddely'}, {'n14': 'rst_n'}, [{'net0': 'nnnn'}, {'Z': '/zzzz/asdasd00'}, {'A': '/gggg/asdsa'}], BUFFER, <netlist.instance object at 0x7efbf1ebf700>, <netlist.instance object at 0x7efbf1b524f0>], {'ext_nets': [{'n14': 'rst_n'}], 'subcircuit': [{'net0': 'nnnn'}, {'Z': '/zzzz/asdasd00'}, {'A': '/gggg/asdsa'}], 'instance': [<netlist.instance object at 0x7efbf1b524f0>]})

In [394]:
x = result1[0]

In [395]:
x.append(1)

AttributeError: 'dict' object has no attribute 'append'

In [396]:
x.pop(2)

KeyError: 2

In [397]:
x.getName()

AttributeError: 'dict' object has no attribute 'getName'

In [398]:
# for ele in result1:
#     print(ele.get_name())
#     print(ele)
#     if ele.get_name() == 'subcircuit':
#         for _ in ele:
#             if type(_) != str:
#                 print(_.get_name())
#                 print(_)
#                 if _.get_name() == "subnetlist":
#                     for t in _:
#                         # print(t.get_name())
#                         print(t)
#     print('\n')
for ele in result1:
    print(ele)

{'n12': 'xcvr_avdd_clk'}
{'n13': 'loaddely'}
{'n14': 'rst_n'}
[{'net0': 'nnnn'}, {'Z': '/zzzz/asdasd00'}, {'A': '/gggg/asdsa'}]
subcircuit BUFFER({'A': BUFFER.A, 'Z': BUFFER.Z, 'gnd': BUFFER.gnd, 'gnds': BUFFER.gnds, 'vdd': BUFFER.vdd, 'vdds': BUFFER.vdds, 'n1': BUFFER.n1, 'n2': BUFFER.n2, 'n3': BUFFER.n3, 'n4': BUFFER.n4, 'n5': BUFFER.n5, 'net0': BUFFER.net0}):{'M1': 'mypmos', 'M2': 'mypmos', 'M3': 'mynmos', 'M4': 'mynmos'}
instance I1@BUFFER{}
instance I2@INVERTER{}


In [367]:
for _ in ele:
    print(_.get_name())

TypeError: 'instance' object is not iterable

In [368]:
_

ParseResults([ParseResults(['net0', 'nnnn'], {'key': 'net0', 'value': 'nnnn'}), ParseResults(['Z', '/zzzz/asdasd00'], {'key': 'Z', 'value': '/zzzz/asdasd00'}), <netlist.instance object at 0x7efbf1467070>, ParseResults(['A', '/gggg/asdsa'], {'key': 'A', 'value': '/gggg/asdsa'}), <netlist.instance object at 0x7efbf1467b50>, <netlist.instance object at 0x7efbf13ecfa0>, <netlist.instance object at 0x7efbf13ec970>], {'int_nets': {'key': 'A', 'value': '/gggg/asdsa'}, 'instance': [<netlist.instance object at 0x7efbf13ec970>]})

In [369]:
_.get_name()

'subnetlist'

In [370]:
ele.get_name()

AttributeError: 'instance' object has no attribute 'get_name'

In [180]:
d = result1.as_dict()

In [181]:
for ele in result1:
    print(ele.as_dict())

{'value': 'xcvr_avdd_clk', 'key': 'n12'}
{'value': 'loaddely', 'key': 'n13'}
{'value': 'rst_n', 'key': 'n14'}
{'name': 'BUFFER', 'nets': {'net': 'n5'}, 'subnetlist': {'int_nets': {'key': 'A', 'value': '/gggg/asdsa'}, 'instance': {'instname': 'M4', 'instbody': ['net0', 'A', 'gnd', 'gnds', 'mynmos', 'w=2', 'l=1']}}}
{'instname': 'I1', 'instbody': ['net0', 'net1', '0', 'gnds!', 'vdd!', 'vdds!', 'BUFFER']}
{'instname': 'I2', 'instbody': ['net1', 'net2', '0', 'gnds!', 'vdd!', 'vdds!', 'INVERTER']}
